In [1]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import subprocess
from pathlib import Path
import configparser
from scipy.stats import ranksums
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.plotting import add_at_risk_counts
from statsmodels.stats import multitest as multi
import plotly.express as px
import plotly.graph_objects as go 
import shutil

#######################################################################################
#                              Carica il file di configurazione                       #
#######################################################################################

config = configparser.ConfigParser()
#script_dir = Path(__file__).parent

# Costruisci il percorso relativo al file di configurazione
config_file ='/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/webserver/conf.ini'

config.read(config_file) 
#directory base
output_data = config['Paths']['output_data']
base_dir = config.get('Paths', 'base_dir', fallback='')

# Costruisci i percorsi completi
def get_full_path(relative_path):
    return os.path.join(base_dir, relative_path)

# Accesso ai valori e costruzione dei percorsi
if 'miRNA' in config:
    miRNA_name = get_full_path(config['miRNA']['name'])
    miRNA_dataframe = get_full_path(config['miRNA']['dataframe'])

if 'gene' in config:
    gene_name_ENSG = get_full_path(config['gene']['name_ENSG'])
    gene_dataframe = get_full_path(config['gene']['dataframe'])
    gene_dataframe_FPKM = get_full_path(config['gene']['dataframe_FPKM'])

if 'protein' in config:
    protein_name = get_full_path(config['protein']['name'])
    protein_dataframe = get_full_path(config['protein']['dataframe'])

if 'clinical' in config:
    clinical_data= get_full_path(config['clinical']['dati_clinici'])
    dati_age= get_full_path(config['clinical']['dati_age'])

if 'os' in config:
    os_pathway=get_full_path(config['OS']['os_pathway'])
    


/Users/chiaranotturnogranieri/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:


def gene_mirna_proteina(gene, cartella):
    #controllo se aprire df gene
    if gene[:4]=='ENSG':
        geni=open("/mnt/data/notturno/gene_expression/ENSG.txt").read().split("\n")
        for g in geni:
            if gene in g:
                gene=g
                os.mkdir(cartella)
                listageni=open("/mnt/data/notturno/gene_expression/ENSG.txt").read().strip().split("\n")
                posizione="-e "+str(listageni.index(gene)+1)+"p"
                
                #creiamo un dataframe piu piccolo dove c'è solo la riga del gene che è stato selezionato
            
                path=cartella+'/'+gene+"_df.txt"
                out_file=open(path,"w")
                subprocess.call(["sed","-n", "-e 1p", posizione, "/mnt/data/notturno/gene_expression/DataFrameTCGA_gene.csv"],stdout=out_file)
                
                return(['gene','gene_id',path,gene])


    #controllo df miRNA
    if gene in open("/mnt/data/notturno/miRNA/namemirna.txt").read().split("\n"):
        os.mkdir(cartella)
        return(['miRNA',"miRNA_ID",'/mnt/data/notturno/miRNA/DataFrameTCGA_miRNA.csv',gene])
    
    
    
    #controllo df proteina
    if gene in open("/mnt/data/notturno/protein/namepeptide.csv").read().split("\n"):
        os.mkdir(cartella)
        return(['protein', "peptide_target",'/mnt/data/notturno/protein/DataFrameTCGA_protein.csv',gene])
    
    else:
        return(0) #la ricerca non è disponibile per il nome inserito



def open_dataframe_gene(gene,listanomi01, path_dataframe):
    if gene == 'miRNA':
        df=pd.read_csv(path_dataframe,usecols=listanomi01)
        df=df.set_index('miRNA_ID')
        return(df)
    if gene == 'gene':
        df=pd.read_csv(path_dataframe,usecols=listanomi01)
        df=df.set_index("gene_id")
        return (df)
    if gene =='protein':
        df=pd.read_csv(path_dataframe,usecols=listanomi01)
        df=df.set_index('peptide_target')
        return (df)
    else: 
        print("per il nome inserito non è disponibile la ricerca")
        return 0



def box_plot(df1, cartella, gene, feature, ogg):
    
    sns.set(rc={'figure.figsize':(25.7,8.27)})
    sns.set_style("white")

    my_order = df1.groupby(by=["tumor"])[gene].median().iloc[::-1].sort_values().index

    ax=sns.boxplot(x="tumor", y=gene, hue=feature, data=df1, palette="Set2", width=0.7, order=my_order)
    
    if ogg == 'miRNA':
        ax.set_yscale("log")
    
    plt.savefig(cartella+'/'+gene+'_'+feature+'.jpg')


def df_feature_age(x):
    if feature=="age_at_initial_pathologic_diagnosis":
        df=pd.read_csv('/mnt/data/notturno/mediana_age_tumor.csv')
        df=df.set_index('acronym')
        
        lista_age=[]
        x1=x
        x1 = x1.drop(x1[x1[feature]== '[Not Available]'].index)

        for tumor,age in zip(x1['acronym'],x1[feature]):
            age=int(age)
            mediana=df.loc[tumor,'median_age']
            
            if age<=mediana:
                lista_age.append("under")
            else:
                lista_age.append("over")

        x1=x1.rename(columns={"age_at_initial_pathologic_diagnosis":"age"})
        x1[feature]=lista_age

        return(x1) 


#per la feature patient status
def crealista_patient(dffeat,listamiRNA,ogg):
    lista=list(dffeat.index)
        
    listaf01=[ogg[1]]
    listaf=[]
    for ele in listamiRNA:
        if ele[:-4] in lista and ele[-1]!="x" and ele[-1]!='y':
            listaf01.append(ele)
            listaf.append(ele[:-4])
            
    
    return listaf01,listaf

def pulisci_df(df):
    for tumor in set(list(df.tumor)):
        df1=df
        df1_mask=df['tumor'] == tumor
        df1=df1[df1_mask]

        lista_feature=((list(df1[feature])))
        t=0
        c=0
        for ele in lista_feature:
            if ele== 'Tumor':
                t+=1
            else:
                c+=1       

        #print(tumor, t, c)
        if t==0 or c==0:
            df.drop(df.index[df['tumor'] == tumor], inplace = True)
            
    return df

#ranksum test per p-value
def p_value(df, cartella):
    tumori=set(list(df.tumor))
    p_values=[]
    t=[]
    for tumor in tumori:
        t.append(tumor)
        p=list(set(df[feature]))

        df1=df
        df1_mask=df1['tumor'] == tumor
        df1=df1[df1_mask]


        filtered_df = df1[df1[feature].eq(p[0])]
        list0= list(filtered_df[gene])


        filtered_df = df1[df1[feature].eq(p[1])]
        list1= list(filtered_df[gene])

        w, p = ranksums(list0, list1)
        p_values.append(p)
    
    df=pd.DataFrame({'tumor':t, 'p-value':p_values})
    df=df.dropna()     
    pvalue_adj=statsmodels.stats.multitest.fdrcorrection(list(df['p-value']), alpha=0.05, method='indep', is_sorted=False)[1]
    df['Adjusted_p-value']= pvalue_adj
    df=df.set_index('tumor')
    df.to_csv(cartella+'/result.txt')
    return df 


path="/mnt/data/notturno/"

gene= sys.argv[1]
feature= sys.argv[2]

cartella=sys.argv[3]


x=pd.read_csv("/mnt/data/notturno/clinical_PANCAN_patient_with_followup_modificato.csv")
x=x.set_index("bcr_patient_barcode")


if feature=="age_at_initial_pathologic_diagnosis":
    x1=df_feature_age(x)
else:
    x1=x


#aprire df
ogg_analisi=gene_mirna_proteina(gene, cartella) #es. miRNA, miRNA_ID, 'pathdataframe.csv'
gene=ogg_analisi[3]
if ogg_analisi!=0:

    #lista samples df expression gene/miRNA/prot
    dd=pd.read_csv(ogg_analisi[2],nrows=1)
    listamiRNA=list(dd.columns[1:])


    #patient status
    if feature== 'patient_status':    
        listanomi01, listanomi= crealista_patient(x, listamiRNA,ogg_analisi)
        
        #aprire df espressione
        d=open_dataframe_gene(ogg_analisi[0],listanomi01, ogg_analisi[2])
        

        lista_exp=[]#lista valori espressione
        lista_feature=[]#lista valori feature
        for ele in d.columns:
            lista_exp.append(d.loc[gene,ele])
            if int(ele[-3:-1])<11:
                lista_feature.append('Tumor')
            else:
                lista_feature.append('Ctrl')
                

    else:
        #list samples with clinical data
        listap=list(x1.index) 
        
        listanomi=[] #nomi con cui ricavare i dati clinici
        listanomi01=[ogg_analisi[1]] #nomi con cui ricavare dati di espressione
        for el in listamiRNA:
            if el[:-4] in listap and el[:-4] not in listanomi:
                if int(el[-3:-1])<11:
                    listanomi.append(el[:-4])
                    listanomi01.append(el)


        #df espressione solo dei campioni di cui abbiamo i dati clinici
        d=open_dataframe_gene(ogg_analisi[0],listanomi01, ogg_analisi[2])



    colonna1=list(d.loc[gene,])
    colonna2=list(x1.loc[listanomi, "acronym"])
    if feature== 'patient_status':
        colonna3=lista_feature
    else:
        colonna3=list(x1.loc[listanomi, feature])


    df=pd.DataFrame({gene:colonna1, 'tumor':colonna2, feature:colonna3})
    df=df.dropna()


    #rimuovere i tumori che non hanno il ctrl 
    if feature=='patient_status':
        df=pulisci_df(df)  



    box_plot(df, cartella, gene, feature, ogg_analisi[0])


    p_value(df, cartella)

else:
    print('Ricerca non disponibile per il nome inserito')
